# [중간보고서] BIM/BM25 기반 한국어 검색 시스템 구현 및 모델 비교 분석

2170045 컴퓨터공학과 서자영

---

## 1. 프로젝트 개요

본 프로젝트는 정보 검색(IR)의 대표적인 확률 모델인 BIM(Binary Independence Model)과 BM25를 직접 구현하고, 한국어 데이터셋(KomuRetrieval)을 활용하여 두 모델의 성능 차이를 비교 분석하는 것을 목표로 한다.

중간 보고 시점에서는 전체 파이프라인(전처리-색인-검색-평가)의 구현을 완료하였으며, 10,000개 샘플 데이터로 시스템을 검증한 후, 50,222개 전체 데이터셋에 대한 성능 평가를 수행하였다.

---

## 2. 데이터셋 및 실험 환경

* **데이터셋 (KomuRetrieval):**
    * **Corpus:** 총 50,222개 문서 (샘플링 실험: 10,000개 / 전체 실험: 50,222개)
    * **Queries:** 1,454개 질의 집합 사용
* **개발 환경:**
    * **Language:** Python 3.14.0
    * **Tools:** Jupyter Notebook
    * **Libraries:** kiwipiepy (형태소 분석), SQLite (역색인 저장), Matplotlib (시각화)
* **실험 파일:**
    * 샘플 실험 결과: `results/search_results_sample10000.json`
    * 전체 실험 결과: `results/search_results.json`

---

## 3. 시스템 구현 (System Implementation)

### 3.1 전처리 및 토큰화 (Preprocessing)

* **형태소 분석:** Kiwi 형태소 분석기를 사용하여 명사(NNG, NNP), 동사(VV), 형용사(VA), 부사(MAG) 등 실질 형태소만 추출하였다.
* **현황:** 중간 단계에서는 별도의 정규화나 불용어 처리를 적용하지 않은 **Baseline Tokenization**을 수행하여, 원본 데이터의 특성을 파악하는 데 주력하였다.
* **토큰화 파이프라인:**
1. **NULL 문자 제거:** SQLite 호환성을 위해 `\x00` 문자를 사전 제거한다.
2. **형태소 분석:** Kiwi 엔진을 통해 텍스트를 형태소 단위로 분해한다.
3. **품사 필터링:** 검색에 유의미한 품사만 선별한다.
   * **NNG/NNP** (일반명사/고유명사): 개체명 및 핵심 개념
   * **VV/VA** (동사/형용사): 행위 및 속성 표현
   * **MAG** (부사): 정도 및 방식 표현
4. **길이 필터링:** 1음절 토큰을 제거하여 노이즈를 감소시킨다.

**구현 코드:**
```python
kiwi = Kiwi(num_workers=-1)  # Windows 호환성을 위한 단일 스레드 모드

def tokenize(text: str) -> List[str]:
    if not text: return []
    clean_text = text.replace('\x00', '')
    try:
        tokens = kiwi.tokenize(clean_text)
        useful_tags = ['NNG', 'NNP', 'VV', 'VA', 'MAG']
        return [t.form for t in tokens if t.tag in useful_tags and len(t.form) > 1]
    except:
        return []
```

### 3.2 시스템 아키텍처

**SQLite**를 사용하여 대용량 데이터 처리와 메모리 효율성을 동시에 확보하였다.

**데이터베이스 스키마:**
1. **documents:** 문서 메타데이터 및 토큰 리스트 저장
   * `doc_id` (PK), `title`, `length`, `tokens` (JSON)
2. **inverted_index:** 단어 → 문서 매핑 정보
   * `term`, `doc_id` (복합 PK), `tf` (단어 빈도)
3. **term_stats:** 단어별 문서 빈도(DF)
   * `term` (PK), `df` (Document Frequency)
4. **statistics:** 전역 통계
   * 총 문서 수(N), 평균 문서 길이(avgdl), 전체 단어 수

### 3.3 역색인(Inverted Index) 구축

**1: 문서 처리 및 순방향 색인 생성**
1) 각 문서의 제목과 본문을 결합하여 토큰화한다.
2) 문서 길이와 토큰 리스트를 `documents` 테이블에 저장한다.
3) 메모리에 `inverted_index` 딕셔너리를 구축한다:
```
   inverted_index[term][doc_id] = term_frequency
```
4) **배치 커밋:** 1,000개 문서마다 DB에 커밋하여 메모리 오버플로우를 방지한다.

**2: 역색인 영구 저장**
1) 메모리 상의 `inverted_index`를 순회하며 DB에 저장한다.
2) 각 단어의 DF(문서 빈도)를 계산하여 `term_stats` 테이블에 저장한다.
3) **배치 커밋:** 10,000개 항목마다 커밋하여 I/O 효율을 최적화한다.

**3: 통계 계산 및 인덱싱**
1) 전역 통계(N, avgdl)를 계산하여 `statistics` 테이블에 저장한다.
2) B-Tree 인덱스를 생성하여 검색 속도를 최적화한다:
   * `CREATE INDEX idx_term ON inverted_index(term)`
   * `CREATE INDEX idx_doc_id ON inverted_index(doc_id)`

### 3.4 검색 알고리즘 구현

* **BIM:** 단어의 출현 여부(Binary)와 IDF 가중치만을 합산하여 점수를 산출한다.
* **BM25:** 단어 빈도(TF)의 포화도($k_1=1.2$)와 문서 길이 정규화($b=0.75$)를 수식에 반영하여 점수를 산출한다.

---

## 4. 결과 및 분석

### 4.1 샘플 실험 (10,000개 문서)

![샘플 실험 결과](../analysis/basic_analysis_sample.png)

#### 정량적 평가 결과
| 모델 | P@5 | P@10 | R@10 | F1@10 |
|------|-----|------|------|-------|
| BIM  | 0.308 | **0.208** | 0.604 | 0.275 |
| BM25 | 0.433 | **0.277** | 0.741 | 0.357 |

* **P@10 기준:** BM25가 BIM 대비 **+33.2% 향상** (0.208 → 0.277)
* **해석:** 샘플 단계에서도 BM25의 TF-IDF 가중치와 문서 길이 정규화가 유효하게 작동함을 확인하였다.

### 4.2 전체 실험 (50,222개 문서)

![전체 실험 결과](../analysis/basic_analysis_full.png)

#### 정량적 평가 결과
| 모델 | P@5 | P@10 | R@10 | F1@10 |
|------|-----|------|------|-------|
| BIM  | 0.207 | **0.147** | 0.461 | 0.199 |
| BM25 | 0.321 | **0.212** | 0.613 | 0.280 |

* **P@10 기준:** BM25가 BIM 대비 **+44.2% 향상** (0.147 → 0.212)
* **해석:** 전체 데이터로 확장 시, 두 모델 모두 절대 성능은 하락하였으나 **BM25의 성능 우위는 더욱 확대**되었다.

### 4.3 분석

#### 주요 발견사항

1. **성능 하락:** (샘플 → 전체)
   * BIM: 0.208 → 0.147 (-29.3%)
   * BM25: 0.277 → 0.212 (-23.5%)
   * 문서 수가 5배 증가하면서 검색 난이도가 상승하였으므로 전체적인 성능은 다소 하락했으나, BIM에 비하면 두 경우 모두 BM25가 더 안정적인 성능을 유지하였다.

2. **상대 성능 격차 확대:**
   * 샘플: +33.2% 향상
   * 전체: +44.2% 향상
   * 대용량 데이터에서 BM25의 **문서 길이 정규화** 효과가 더욱 두드러졌다.

3. **Recall 차이:**
   * 샘플 R@10: BIM 0.604 vs BM25 0.741
   * 전체 R@10: BIM 0.461 vs BM25 0.613
   * BM25는 관련 문서를 더 많이 상위 랭킹에 포함시키는 능력이 우수한 것으로 보인다.

4. **BIM vs. BM25**
   * 전체 데이터셋 기준, BM25 모델이 BIM 대비 약 44.2% 높은 정밀도를 기록하였다. 이는 한국어 정보 검색 환경에서 단순한 단어의 유무(Presence)보다는, 단어의 빈도(Term Frequency)와 문서의 길이(Document Length) 정보가 검색 랭킹 산정에 결정적인 역할을 한다고 볼 수 있다.

---

## 5. 기술적 이슈 및 해결 (Troubleshooting)

1. **Windows 멀티프로세싱 충돌:** Kiwi 형태소 분석기 구동 시 `UnicodeDecodeError`가 발생하여, `num_workers=-1` 옵션을 통해 단일 스레드 모드로 전환하여 안정성을 확보하였다.

2. **파일 잠금(File Locking) 오류:** Jupyter Notebook 재실행 시 DB 파일 삭제(`os.remove`) 권한 오류가 발생하여, SQL의 `DROP TABLE` 구문을 사용하여 연결을 유지한 채 테이블만 초기화하도록 로직을 개선하였다.

3. **메모리 효율화:** 대용량 데이터 로드 시 발생하는 메모리 부하를 줄이기 위해 `Batch Processing`과 `Incremental Commit` 방식을 적용하였다.

---

## 6. 향후 계획: 데이터 특성 기반 심층 분석 (Advanced Analysis)

최종 보고서에서는 단순한 평균 점수 비교를 넘어, '데이터의 구조적 특성(Structural Characteristics)이 모델 성능에 미치는 영향'을 규명하기 위해 다음과 같은 세분화 평가(Segmented Evaluation)를 수행할 예정이다.

### 6.1 데이터 기반 후처리 (Data-Driven Preprocessing)

* **현황 분석:** 낮은 모델 성능으로 인해 EDA를 진행한 결과, `나오다`, `경우` 등 상위 빈도 어휘가 전체 토큰의 상당수를 점유하고 있으며, 이들의 IDF 값이 0.5 미만으로 낮아 검색 변별력이 없음을 확인하였다.
* **개선 계획:**
    * **불용어(Stopwords) 최적화:** 50,222개 문서 전수 조사를 통해 식별된 저빈도 정보어(Low IDF Words)를 제거하여 인덱스 효율성을 높인다.
    * **노이즈 정제:** 나무위키 문서 특유의 비격식/비정형 텍스트(자모음 `ㅋㅋ`, 반복 부호 `......` 등)를 정규표현식으로 정제하여, 불필요한 토큰 생성을 방지하고 BM25의 길이 정규화 왜곡을 최소화한다.

### 6.2 도메인(특성)별 세분화 평가 (Deep Analysis)

문서를 단순한 개수가 아닌 **언어학적 단위(음절, 형태소, 문장)**로 분석하여 모델의 유리/불리함을 파악한다.

* **문서 길이(Length) 분석:**
    * 문서를 형태소 개수(`morph_count`) 기준으로 Short / Medium / Long 그룹으로 분할한다.
    * 길이가 긴 문서에서 발생하는 스코어 편향을 BM25의 길이 정규화 파라미터($b$)가 얼마나 효과적으로 보정하는지 검증한다.
* **문장 복잡도(Complexity) 분석:**
    * 실러블(음절) 수와 Kiwi 문장 분리기를 활용하여, 문서당 **'평균 문장 길이(음절 수 / 문장 수)'**를 측정한다.
    * 문장의 호흡이 길고 구조가 복잡한(Complex) 문서군에서 키워드 빈도(TF)가 검색 성능에 미치는 영향을 분석한다.
* **질의 유형(Query Type) 분석:**
    * 검색 질의를 토큰 길이에 따라 **Keyword형(1~2단어)**과 **Verbose형(3단어 이상)**으로 분류한다.
    * 사용자의 검색 의도가 구체적일수록(Verbose) BM25 가중치 산정 방식의 우수성을 입증한다.

### 6.3 최종 산출물 (예정)

* 5만 개 전체 데이터 기반의 성능 비교 보고서 (전처리 전/후 비교 포함).
* 도메인별(길이, 복잡도, 질의) 상세 분석 그래프.
* 검색 데모 UI 구현.